In [1]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def _gini(self, y):
        classes = np.unique(y)
        gini = 1.0
        for c in classes:
            p = np.sum(y == c) / len(y)
            gini -= p ** 2
        return gini

    def _best_split(self, X, y):
        best_feat, best_thresh = None, None
        best_gini = 1.0
        n_samples, n_features = X.shape

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for thresh in thresholds:
                left = y[X[:, feature] <= thresh]
                right = y[X[:, feature] > thresh]
                if len(left) == 0 or len(right) == 0:
                    continue
                gini_left = self._gini(left)
                gini_right = self._gini(right)
                weighted_gini = (len(left)/n_samples)*gini_left + (len(right)/n_samples)*gini_right

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feat = feature
                    best_thresh = thresh

        return best_feat, best_thresh

    def _build_tree(self, X, y, depth=0):
        if (depth >= self.max_depth or len(np.unique(y)) == 1 or len(y) < self.min_samples_split):
            return np.bincount(y).argmax()

        feat, thresh = self._best_split(X, y)
        if feat is None:
            return np.bincount(y).argmax()

        left_idx = X[:, feat] <= thresh
        right_idx = X[:, feat] > thresh

        left_child = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right_child = self._build_tree(X[right_idx], y[right_idx], depth + 1)

        return (feat, thresh, left_child, right_child)

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _predict_one(self, x, tree):
        if not isinstance(tree, tuple):
            return tree
        feat, thresh, left, right = tree
        if x[feat] <= thresh:
            return self._predict_one(x, left)
        else:
            return self._predict_one(x, right)

    def predict(self, X):
        return np.array([self._predict_one(sample, self.tree) for sample in X])


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tree = DecisionTree(max_depth=3)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9555555555555556
